In [1]:
from kiteconnect import KiteConnect
from kiteconnect import exceptions

In [2]:
import logging
import requests
import ctypes

In [3]:
import pandas as pd

In [4]:
from time import time, sleep

In [5]:
from datetime import datetime, timedelta

In [6]:
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [8]:
api_key = "f5ean4aaxq3wv9n3"
client_id = "ED5107"
api_secret = "s1kbs5y654zhbpqgd2rvslqc6shx9yh8"

In [9]:
logging.basicConfig(level=logging.DEBUG)
kite = KiteConnect(api_key=api_key)
print(kite.login_url())

https://kite.trade/connect/login?api_key=f5ean4aaxq3wv9n3&v=3


In [10]:
# Get the Request Token from the redirect URL

request_token = "kxUB29orXpZDVOCpARctWElF0bPoy54t"

# Get Access Token

In [11]:
#data = kite.generate_session(request_token, api_secret=api_secret)
#access_token = data["access_token"]
access_token = "ngIWpCT8mnlRmnBiSV5T32C7EJKZql5E"
kite.set_access_token(access_token)
print(access_token)

ngIWpCT8mnlRmnBiSV5T32C7EJKZql5E


In [12]:
print(kite.profile())

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /user/profile HTTP/1.1" 200 None


{'user_id': 'ED5107', 'user_type': 'individual', 'email': 'ramanathannalini@gmail.com', 'user_name': 'Nalini Ramanathan', 'user_shortname': 'Nalini', 'broker': 'ZERODHA', 'exchanges': ['BFO', 'CDS', 'BSE', 'NFO', 'MF', 'NSE'], 'products': ['CNC', 'NRML', 'MIS', 'BO', 'CO'], 'order_types': ['MARKET', 'LIMIT', 'SL', 'SL-M'], 'avatar_url': '', 'meta': {'demat_consent': 'consent'}}


# Get Instrument Tokens

In [13]:
def get_instrument_token(tradingsymbol, instrument_type, exchange):
    instruments = pd.DataFrame(kite.instruments())
    instrument_token = instruments[(instruments['tradingsymbol']==tradingsymbol) & (instruments['instrument_type']==instrument_type) & (instruments['exchange']==exchange)]['instrument_token'].to_list()[0]
    return str(instrument_token)

# Get Historical Data

In [14]:
def get_historical_data(instrument_token, from_date, to_date, interval):
    return pd.DataFrame(kite.historical_data(instrument_token=instrument_token,
                                    from_date=from_date, 
                                   to_date=to_date,
                                   interval=interval))

# Get Exponential Moving Average

In [15]:
def get_exponential_moving_avg(timeseries, span):
    """
    adjust=False specifies that we are interested in the recursive calculation mode.
    """
    timeseries[str(span)+'d_EMAvg'] = timeseries.iloc[:,0].ewm(span=span, adjust=False).mean()
    return timeseries

# Stochastic Momentum Indicator

In [16]:
def get_smi(hist_data, k_period = 10, k_smooth_period = 3, d_smooth_period = 3, d_period = 10):
    close = pd.DataFrame({'date':[datetime.strftime(d, "%Y-%m-%d %H:%M:%S") for d in list(hist_data['date'])],'close':list(hist_data['close'])})
    highest_high = pd.DataFrame(data=list(hist_data['high']), columns=['highest_high']).rolling(k_period, min_periods=1).max()
    lowest_low = pd.DataFrame(data=list(hist_data['low']), columns=['lowest_low']).rolling(k_period, min_periods=1).min()
    mean_hl = pd.DataFrame(data=(highest_high['highest_high']+lowest_low['lowest_low'])*0.5, columns=['mean_hl'])
    hh_minus_ll = pd.DataFrame(data=(highest_high['highest_high']-lowest_low['lowest_low']), columns=['hh_minus_ll'])
    close_minus_mean_hl = pd.DataFrame(data=(close['close']-mean_hl['mean_hl']), columns=['close_minus_mean_hl'])
    emavg_cmmhl = pd.DataFrame(data=list(close_minus_mean_hl['close_minus_mean_hl'].ewm(span=k_smooth_period, adjust=False).mean()), columns=['emavg_cmmhl'])
    emavg_emavg_cmmhl = pd.DataFrame(data=list(emavg_cmmhl['emavg_cmmhl'].ewm(span=k_smooth_period, adjust=False).mean()), columns=['emavg_emavg_cmmhl'])
    emavg_hhmll = pd.DataFrame(data=list(hh_minus_ll['hh_minus_ll'].ewm(span=d_smooth_period, adjust=False).mean()), columns=['emavg_hhmll'])
    emavg_emavg_hhmll = pd.DataFrame(data=list(emavg_hhmll['emavg_hhmll'].ewm(span=d_smooth_period, adjust=False).mean()), columns=['emavg_emavg_hhmll'])
    smi = pd.DataFrame(data=100*(emavg_emavg_cmmhl['emavg_emavg_cmmhl']/(emavg_emavg_hhmll['emavg_emavg_hhmll']/2)), columns=['smi'])
    signal = pd.DataFrame(data=list(smi['smi'].ewm(span=d_period, adjust=False).mean()), columns=['signal'])
    smi_df = pd.concat([close, smi, signal],axis=1)
    smi_df.set_index('date', drop=True, inplace=True)
    return smi_df

# Zerodha Testing LIVE Deployment TradeLog

In [17]:
def get_quote(exchange, tradingsymbol):
    for _ in range(5):
        try:
            quote = kite.quote(exchange+":"+tradingsymbol)[exchange+":"+tradingsymbol]
            break
        except requests.exceptions.ReadTimeout as e:
            logging.exception("Timeout Error - while getting ltp: \n %s", e)
            logging.info("Retrying")
            sleep(1)
            continue
    return quote

In [18]:
def create_candle(price, latest_candle, candle_data, all_prices, first_candle_timestamp, candle):
    # Add latest price to latest_candle data
    latest_candle.loc[len(latest_candle)] = price
    
    # get OHLC for latest candle
    price_list = list(latest_candle['price'])
    actual_high = max(price_list)
    actual_low = min(price_list)
    actual_close = price_list[-1]
    actual_open = price_list[0]
    
    # get Heikin Ashi candle OHLC data
    ha_close = (1/4)*(actual_open+actual_low+actual_high+actual_close)
    if len(candle_data)==0:
        ha_open = (prev_ha_open+prev_ha_close)/2
    else:
        ha_open = (list(candle_data['Open'])[-1] + list(candle_data['Close'])[-1])/2
    ha_high = max([actual_high, ha_open, ha_close])
    ha_low = min([actual_low, ha_open, ha_close])
    if ha_open < ha_close:
        color = 'green'
    else:
        color = 'red'
    
    # Add to candle data
    candle_data.loc[len(candle_data)] = [first_candle_timestamp, ha_open, ha_high, ha_low, ha_close, color]
    all_prices = pd.concat([all_prices, latest_candle])
    
    # Update variables for next candle
    first_candle_timestamp = first_candle_timestamp+timedelta(minutes=candle)
    latest_candle = pd.DataFrame(columns=['price'])
    
    return latest_candle, candle_data, all_prices, first_candle_timestamp, color, price_list

In [19]:
def its_transaction_time(color, first_candle_timestamp, price_list, current_status, current_transaction_type, trade_log):
    # Use candle data to enter trade
    if color=='red':
        transaction_type = 'sell'
        if current_status=='open':
            if current_transaction_type == 'buy':
                trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=first_candle_timestamp, price=price_list[-1], open_price=price_list[0], current_transaction_type=transaction_type)
                trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=first_candle_timestamp, price=price_list[-1], open_price=price_list[0], current_transaction_type=transaction_type)
        elif current_status=='closed':
            trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=first_candle_timestamp, price=price_list[-1], open_price=price_list[0], current_transaction_type=transaction_type)
    elif color=='green':
        transaction_type = 'buy'
        if current_status=='open':
            if current_transaction_type == 'sell':
                trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=first_candle_timestamp, price=price_list[-1], open_price=price_list[0], current_transaction_type=transaction_type)
                trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=first_candle_timestamp, price=price_list[-1], open_price=price_list[0], current_transaction_type=transaction_type)
        elif current_status=='closed':
            trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=first_candle_timestamp, price=price_list[-1], open_price=price_list[0], current_transaction_type=transaction_type)
    return trade_log, transaction_price, current_transaction_type, current_status

In [20]:
def place_trade(trade_log, current_time, price, open_price, current_transaction_type):
    if len(trade_log)==0 or trade_log.loc[list(trade_log.index)[-1], 'status']=='closed':
        current_status = 'open'
    elif trade_log.loc[list(trade_log.index)[-1], 'status']=='open':
        current_status = 'closed'
    if price>open_price:
        candle_color = 'green'
    else:
        candle_color = 'red'
    print(current_time, current_transaction_type, price, current_status)
    trade_log.loc[len(trade_log)] = [current_time, open_price, price, candle_color, current_transaction_type, current_status]
    return trade_log, price, current_transaction_type, current_status

# Pre-Flight Checks

### WARNING: Don't forget to update the date and time in 'deply_till' variable

In [21]:
#INSTRUMENT DATA
tradingsymbol = 'BANKNIFTY20SEPFUT'
instrument_type = 'FUT'
exchange = 'NFO'

# TRADE DETAILS
qty = 1
order_type='MARKET'
product='MIS'
variety='regular'

# CHART SPECIFICATIONS
candle_size = '3minute'
k_period = 10
k_smooth_period = 3
d_smooth_period = 3
d_period = 10

# DEPLOYMENT DURATION
deploy_till = datetime(year=2020, month=8, day=28, hour=15 , minute=30 , second=0)

In [22]:
candle = 3
target = 20
buffer = 5
stop_loss = -100
first_candle_timestamp = datetime(2020, 8, 28, 12, 9)

prev_ha_open = 24586.29
prev_ha_close = 24620.04

# THE PAYLOAD

___________________________________________

##### The Algorithm

candle = 5
target = 20
buffer = 5
start = 9:30
next_transaction_time = 9:29:50
current transaction = None
current_status = 'closed'
pnl = 0
n=1

while time.now() != 15:20:
sleep(1)
test_end = now
test_start = now - 20mins
hist_data
price = hist_data.loc[list(smi_df.index)[-1], 'close']
open_price = hist_data.loc[list(smi_df.index)[-1], 'open']
if current transaction == 'buy':
    pnl = price - transaction_price
elif current transaction == 'sell':
    pnl = transaction_price - price
if pnl < -50:
    if prev trade is buy:
        sell to exit
    if prev trade is sell:
        sell to buy
elif pnl >= target:
    while True:
        trailing_stoploss = target - buffer
        ## get latest price
        if current transaction == 'buy':
            pnl = price - transaction_price
        elif current transaction == 'sell':
            pnl = transaction_price - price
        if pnl < trailing_stoploss:
            if prev trade is buy:
                sell to exit
                current_transaction_type = None
            elif prev trade is sell:
                sell to buy
                current_transaction_type = None
            break
        if pnl > trailing stoploss:
            ## include if and elif to add price to candle data
            ## check if end_time has been reached. Close position and exit if yes
            ## if time for next transaction
            ## create candle
            ## change next transaction time
            ## reset latest_candle
            ## update first candle timestamp
    continue


if test_end.minute = next_transaction_time.minute and test_end.second >= next_transaction_time.second:
        if candle red:
            if trade OPEN:
                if existing trade is buy:
                    sell to exit buy
                    sell to take short poition
                if existing trade is sell:
                    do nothing
            else if trade CLOSED:
                sell
        elif candle green:
            if trade OPEN:
                if existing trade is sell:
                    buy to exit sell
                    buy to take long poition
                if existing trade is buy:
                    do nothing
            else trade CLOSED:
                buy
        n+1
        next_transaction_time = start + 5x(n)minutes -10seconds
        sleep 15

________________________________________

In [23]:
instoken = get_instrument_token(tradingsymbol, instrument_type, exchange)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments HTTP/1.1" 200 860315


In [41]:
next_transaction_time = first_candle_timestamp + timedelta(minutes=candle)
trade_log = pd.DataFrame(columns=['time','open_price','price','candle_color','transaction_type','status'])
candle_data = pd.DataFrame(columns=['Candle','Open','High','Low','Close','Color'])
all_prices = pd.DataFrame(columns=['price'])
latest_candle = pd.DataFrame(columns=['price'])
current_transaction_type = None
current_status = 'closed'
pnl = 0

while datetime.strftime(datetime.now(), "%H:%M")!=datetime.strftime(deploy_till-timedelta(minutes=1), "%H:%M"):
    sleep(1)
    timestamp = datetime.now()
    quote = get_quote(exchange, tradingsymbol)
    price = quote['last_price']
    
    if current_status=="closed" and timestamp.minute==next_transaction_time.minute and timestamp.second<(first_candle_timestamp+timedelta(minutes=candle)).minute:
        latest_candle.loc[len(latest_candle)] = price
    elif (first_candle_timestamp+timedelta(minutes=candle)).minute==0 and timestamp.hour<(first_candle_timestamp+timedelta(minutes=candle)).hour:
        latest_candle.loc[len(latest_candle)] = price
    elif timestamp.minute==next_transaction_time.minute and (timestamp.second==0 or timestamp.second==1):
        latest_candle, candle_data, all_prices, first_candle_timestamp, color, price_list = create_candle(price, latest_candle, candle_data, all_prices, first_candle_timestamp, candle)
        if current_status=='closed' or (color=='green' and current_transaction_type!='buy') or (color=='red' and current_transaction_type!='sell'):
            trade_log, transaction_price, current_transaction_type, current_status = its_transaction_time(color, first_candle_timestamp, price_list, current_status, current_transaction_type, trade_log)
        next_transaction_time = first_candle_timestamp + timedelta(minutes=candle)
        pnl=0
        sleep(1)
        continue
    
    if datetime.strftime(timestamp, "%H:%M")==datetime.strftime(deploy_till-timedelta(minutes=2), "%H:%M"):
        if current_status=='open':
            if current_transaction_type=='buy':
                transaction_type='sell'
                trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=first_candle_timestamp, price=price, open_price=list(latest_candle['price'])[0], current_transaction_type=transaction_type)
                break
            if current_transaction_type=='sell':
                transaction_type='buy'
                trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=first_candle_timestamp, price=price, open_price=list(latest_candle['price'])[0], current_transaction_type=transaction_type)
                break
        else:
            break
    
    if current_transaction_type == 'buy':
        pnl = price - transaction_price
    elif current_transaction_type == 'sell':
        pnl = transaction_price - price
    
    if pnl < stop_loss:
        if current_transaction_type == 'buy':
            transaction_type = 'sell'
            trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=first_candle_timestamp, price=price, open_price=list(latest_candle['price'])[0], current_transaction_type=transaction_type)
        elif current_transaction_type == 'sell':
            transaction_type = 'buy'
            trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=first_candle_timestamp, price=price, open_price=list(latest_candle['price'])[0], current_transaction_type=transaction_type)
        current_transaction_type = None
        pnl=0
    elif pnl >= target:
        trailing_stoploss = pnl-buffer
        highest_pnl = pnl
        while True:
            # Get Latest Price
            quote = get_quote(exchange, tradingsymbol)
            price = quote['last_price']
            
            if timestamp.minute>=first_candle_timestamp.minute and timestamp.minute<(first_candle_timestamp+timedelta(minutes=candle)).minute:
                latest_candle.loc[len(latest_candle)] = price
            elif (first_candle_timestamp+timedelta(minutes=candle)).minute==0 and timestamp.hour<(first_candle_timestamp+timedelta(minutes=candle)).hour:
                latest_candle.loc[len(latest_candle)] = price
            elif timestamp.minute==next_transaction_time.minute and (timestamp.second==0 or timestamp.second==1):
                latest_candle, candle_data, all_prices, first_candle_timestamp, color, price_list = create_candle(price, latest_candle, candle_data, all_prices, first_candle_timestamp, candle)
                next_transaction_time = first_candle_timestamp + timedelta(minutes=candle)
            
            # if day close is reached then exit position
            if datetime.strftime(timestamp, "%H:%M")==datetime.strftime(deploy_till-timedelta(minutes=2), "%H:%M"):
                if current_transaction_type=='buy':
                    transaction_type='sell'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=first_candle_timestamp, price=price, open_price=list(latest_candle['price'])[0], current_transaction_type=transaction_type)
                elif current_transaction_type=='sell':
                    transaction_type='buy'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=first_candle_timestamp, price=price, open_price=list(latest_candle['price'])[0], current_transaction_type=transaction_type)
                current_transaction_type = None
                pnl=0
                break
            
            # get PnL
            if current_transaction_type == 'buy':
                pnl = price - transaction_price
            elif current_transaction_type == 'sell':
                pnl = transaction_price - price
            
            if pnl < trailing_stoploss:
                # exit current position
                if current_transaction_type == 'buy':
                    transaction_type = 'sell'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=first_candle_timestamp, price=price, open_price=list(latest_candle['price'])[0], current_transaction_type=transaction_type)
                elif current_transaction_type == 'sell':
                    transaction_type = 'buy'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=first_candle_timestamp, price=price, open_price=list(latest_candle['price'])[0], current_transaction_type=transaction_type)
                current_transaction_type = None
                pnl=0
                break
            
            elif pnl>trailing_stoploss and pnl<highest_pnl:
                continue
            
            elif pnl>highest_pnl:
                # update trailing stoploss and highest pnl
                trailing_stoploss = pnl-buffer
                highest_pnl = pnl

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connec

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connec

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connec

2020-08-28 12:12:00 buy 24402.05 open


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connec

...

2020-08-28 13:45:00 buy 24414.95 open


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connec

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connec

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connec

UnboundLocalError: local variable 'transaction_price' referenced before assignment

In [42]:
trade_log['time'] = [datetime.strftime(d, "%H:%M:%S") for d in list(trade_log['time'])]
trade_log.to_excel('LIVE Deployment Logs\RUN1D-QD-20200828-3min_Candle_reading_prev_candle_close.xlsx')

In [43]:
candle_data.to_excel('Daily_candle_and_price_data\Candles_20200828_B.xlsx')
all_prices.to_excel('Daily_candle_and_price_data\All_prices_20200828_B.xlsx')

In [44]:
pnl

15.049999999999272

In [49]:
target

20

In [47]:
price

24430

In [48]:
transaction_price

24414.95

In [57]:
timestamp

datetime.datetime(2020, 8, 28, 14, 48, 0, 821670)

In [54]:
next_transaction_time

datetime.datetime(2020, 8, 28, 13, 48)

In [52]:
first_candle_timestamp

datetime.datetime(2020, 8, 28, 13, 48)

In [56]:
datetime(2020, 8, 28, 13, 48) + timedelta(minutes=15)

datetime.datetime(2020, 8, 28, 14, 3)

In [59]:
current_status

'open'

In [60]:
current_transaction_type

'buy'